In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colormaps
import numpy as np
from scipy import stats
from sklearn.metrics import r2_score


In [ ]:
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

df = pd.read_csv("data/filmsdata.csv")
df[["saturation"]] = df[["saturation"]].fillna(0)
df1 = df
df2 = df
df
# df["saturation"].describe()
# years = df.groupby("year")
# print(years["imdb_rating"])

#### Aspect Ratio Over Time

In [ ]:
df1.dropna(subset = ["aspect_ratio"], inplace=True)
x = df1["year"]
# print(df["aspect_ratio"])
y = df1["aspect_ratio"]

num_degrees = 2

#Fit model
poly_model = np.polyfit(x, y, num_degrees)
poly_model = np.poly1d(poly_model)

# slope, intercept, r, p, std_err = stats.linregress(x, y)
# slope = np.float32(slope)
# model_y = [slope * i + intercept for i in x]

aspect = y.unique()
print(aspect)
fig, ax = plt.subplots(figsize = (12,8))
ax.scatter(x, y, c = "black", s = 100, alpha = 0.2)
ax.set_title("Aspect Ratio over Time")
# ax.plot(x, model_y, color='red')
model_x = x
model_y = poly_model(model_x)
ax.plot(model_x,model_y,"rx", markersize = 15)
ax.set_yticks(aspect)

#### Rotten Tomato Dataset

some data cleaning was required 

In [ ]:
tomato = pd.read_csv("data/rotten_tomatoes_top_movies.csv")
unique_asp = tomato["aspect_ratio"].unique()
print(unique_asp)
tomato = tomato[tomato["aspect_ratio"].notna()]
import re
for index, row in tomato.iterrows():
    aspect_ratio = row["aspect_ratio"]
    if aspect_ratio == "1.43:1":
        tomato.at[index, "aspect_ratio"] = float(1.43)
    elif aspect_ratio == "1.66:1":
        tomato.at[index, "aspect_ratio"] = float(1.66)
    else:
        temp_asp = re.search(r"(?<=\()(\d+\.\d{1,2})?(?=:)", aspect_ratio)
        if temp_asp is not None:
            aspect_ratio_value = float(temp_asp.group())
            tomato.at[index, "aspect_ratio"] = aspect_ratio_value
        else:
            tomato.drop(index, inplace=True)
# tomato = tomato[tomato["aspect_ratio"].notna()]

unique_asp = tomato["aspect_ratio"].unique()
print(unique_asp)

In [ ]:
test = tomato[tomato["aspect_ratio"] == 1.43]
test

Correlation matrix, people and critics dont agree that much it seems

In [ ]:
tomato = pd.read_csv("data/rotten_tomatoes_top_movies.csv")
corr2 = tomato.corr(numeric_only=True)
corr2.style.background_gradient(cmap='RdYlGn')

In [ ]:
tomato_x = tomato["year"]
tomato['aspect_ratio'] = pd.to_numeric(tomato['aspect_ratio'], errors='coerce')
tomato_y = tomato["aspect_ratio"]
print(tomato_y.dtype)
fig, ax = plt.subplots(figsize = (12,6))
ax.scatter(tomato_x, tomato_y, c = "black", s = 100, alpha = 0.2)
\
ax.set_title("Rotten Tomato Dataset Aspect Ratio over Time")
# ax.plot(x, model_y, color='red')
# ax.tick_params(axis='y', labelrotation=25)
# y_ticks = tomato["aspect_ratio"].unique()
y_ticks = [1.33, 1.37, 1.43, 1.66, 1.85, 2.35]

ax.set_yticks(y_ticks)

#### Average Shot Scale

In [ ]:
df.dropna(subset = ["ave_scale"], inplace=True)
x = df["year"]
# print(df["aspect_ratio"])
y = df["ave_scale"]
# print(x)
# num_degrees = 1

# #Fit model
# poly_model = np.polyfit(x, y, num_degrees)
# poly_model = np.poly1d(poly_model)

slope, intercept, r, p, std_err = stats.linregress(x, y)
slope = np.float32(slope)
model_y = [slope * i + intercept for i in x]

fig, ax = plt.subplots(figsize = (8,5))
ax.scatter(x, y, c = "black", s = 20, alpha = 1)
ax.set_title("Average Shot Scale over Time")
ax.plot(x, model_y, color='red')
ax.set_ylim(2, 5)
y_ticks = range(2, 6)
ax.set_yticks(y_ticks)
scale_labels = {2: "2: Long Shot", 3: "3: Medium Long Shot", 4: "4: Medium Shot", 5: "5: Medium Close Up"}
ax.set_yticklabels([scale_labels.get(tick, '') for tick in y_ticks])
# model_x = x
# model_y = poly_model(model_x)
# ax.plot(model_x,model_y,"rx", markersize = 15)

print(r)


#### Averge Shot Length

In [ ]:
df2.dropna(subset = ["ASL"], inplace=True)
x = df2["year"]
# print(df["aspect_ratio"])
y = df2["ASL"]
fig, ax = plt.subplots(figsize = (10,6))
ax.scatter(x, y, c = "lightseagreen", s = 50, alpha = 0.5 )
ax.set_title("Average Shot Length over Time")

num_degrees = 1

poly_model = np.polyfit(x, y, num_degrees)
poly_model = np.poly1d(poly_model)
model_x = x
model_y = poly_model(model_x)
ax.plot(model_x,model_y,"rx", markersize = 10, color = "darkred")
ax.set_yscale("log", base = 2)
ax.set_ylabel("Shot Length in seconds")
ax.set_ylim(1, 33)
ax.set_yticklabels([0.5, 1, 2, 4, 8, 16, 32])

print(r2_score(y, poly_model(model_x)))

#### Correlation matrix

In [ ]:

corr = df.corr(numeric_only=True)
corr.style.background_gradient(cmap='RdYlGn')

#### IMDB Score Predition 

In [ ]:
rating =  df[df["imdb_rating"].notna()]
# rating = df[df["genre"] == "comedy"]
# rating
fig, ax = plt.subplots(figsize = (12, 6))
year_list = pd.unique(rating["year"])
# print(year_list)
genre_list = pd.unique(rating["genre"])
colors = colormaps['viridis']
color_list = [colors(i) for i in np.linspace(0,1,len(genre_list))]
color_dict = {str(genre_list[i]):color_list[i] for i in range(len(genre_list))}

# ax.scatter(df["year"], df["imdb_rating"], s = 7)
for genre, group in rating.groupby('genre'):
    ax.scatter(group['year'], group['imdb_rating'], s=100, alpha = 0.8, label=genre, color=color_dict[genre])

ax.set_title("IMDB score over time")
ax.set_xticks(year_list)
ax.set_xticklabels(year_list,rotation=25)
ax.legend()

num_degrees = 4
poly_model = np.polyfit(rating["year"], rating["imdb_rating"], num_degrees)
poly_model = np.poly1d(poly_model)
model_x = x
model_y = poly_model(model_x)
ax.plot(model_x,model_y, "x", markersize = 10, color = "red")

print(r2_score(y, poly_model(model_x)))


Label Encoder for genres

In [ ]:
from sklearn.preprocessing import LabelEncoder
df = pd.read_csv("data/filmsdata.csv")
df = df[df["year"].notna()]
df = df[df["imdb_rating"].notna()]

encoded_y = LabelEncoder()
df["genre_encoded"] = encoded_y.fit_transform(df["genre"])
# print(encoded_y)
# print(len(encoded_y), len(df["genre"]))
mapping = dict(zip(encoded_y.classes_, encoded_y.transform(encoded_y.classes_)))
print(mapping)
df["genre_encoded"]


linear regression model

In [ ]:
from sklearn import linear_model 
# from sklearn.metrics import mean_squared_error, r2_score

regr = linear_model.LinearRegression()
regr_x = df[["year", "genre_encoded"]]
regr_y = df["imdb_rating"]
regr.fit(regr_x, regr_y) 
print(regr.coef_, regr.intercept_,regr.score(regr_x, regr_y))
feature_names = regr_x.columns
# print(feature_names)
predict = regr.predict([[3000, 3]])
print(predict)

loop through 50 years and all genres and put predictions into a table

In [ ]:
all_years = range(2025, 2080, 5)
all_genres = df['genre_encoded'].unique()

predictions_df = pd.DataFrame(index=all_years, columns=all_genres)

for year in all_years:
    for genre in all_genres:

        input_features = [[year, genre]]
        prediction = regr.predict(input_features)
        
        predictions_df.at[year, genre] = prediction

swapped_map = {v: k for k, v in mapping.items()}
predictions_df.rename(columns=swapped_map, inplace=True)
predictions_df